# Query Plan (Process for generate the Query plan):
- Takes in all code
  - It check syntax
- generate unresolved logical plan (also called, Parsed Logical Plan) after syntax is correct.
(Internally spark has a structure called the catalog (maintain detail about what are table, dataframe, datatype - check if table, column,  exist in table, it does this in all dataset))
- Generate logical plan (also, Analyzed Logical Plan: once generated it goes through the catalyst optimizer, which is going to do some few optimization, like Filter push-down, push-down projection)
  - Push-down projection: say you have written `select *` but then spark figures out you only used 5 columns in your code, so spark will only select those 5 columnns instead of using `*` which fetches all columns.
  - Filter pushdown: Lets say you are doing a filter in the middle of your query with the DF, spark figures out this filter can be push to the data source itself, when you are querying the datasource, you can combine this filter with the query, so what it does is that it is going to return you small number of rows right at the start, these are some of the optimization it does.
- generate an optimized logical plan (which is then converted to several physical plan)
- Optimized logical plan goes through, Cost-Based Optimizer which then picks the best Physical Plan (This is what is ran on the Cluster)
- That final plan is executed on the cluset.

This is how the Query Plan is generated from start till the end.



## ✅ Why Read the Physical Plan if It’s Optimized?

### 1. **"Optimized" ≠ "Perfect for Your Case"**

* Spark chooses a plan based on general rules and statistics (if available).
* But sometimes, due to:

  * Skewed data
  * Inaccurate/no statistics
  * Small files
  * Legacy formats (like CSV)
  * Bad join keys
* The chosen "best" plan might still perform **poorly in reality**.

> 🔍 **Reading the physical plan helps catch inefficiencies Spark didn’t predict.**

---

### 2. **Spotting Expensive Operations**

From your physical plan:

```text
SortMergeJoin ...
Exchange hashpartitioning(MSISDN, 200)
Sort ...
```

This tells you:

* Spark is **shuffling** both sides of the join (`Exchange`)
* Then **sorting** both sides (`Sort`)
* Then using **SortMergeJoin**

This join works **well for large datasets**, but:

* If data is **already partitioned/sorted**, the sort and shuffle are **wasted work**
* If you know your data is **small**, a **BroadcastHashJoin** would be faster

You might:

```python
from pyspark.sql.functions import broadcast

df1.join(broadcast(df2), "MSISDN")
```

to **override** Spark’s default decision.

---

### 3. **Understand Execution Costs**

Looking at the physical plan helps you estimate:

* How much **shuffling** will occur (expensive)
* How many **tasks/stages** will be created
* Whether **filters were pushed down** to data sources (important for large files)

---

### 4. **Debugging Performance Issues**

If your job is slow or failing:

* Reading the plan helps you know where Spark is **spending time**
* For example:

  * Are joins causing skew?
  * Is filter pushdown working?
  * Are there unnecessary shuffles?

---

## ✅ Summary (short + simple)

| **Reason** to Read Plan    | **Why It Matters**                          |
| -------------------------- | ------------------------------------------- |
| See unnecessary operations | Catch avoidable shuffles, sorts, or scans   |
| Validate optimization      | Ensure Spark picked the *truly* best plan   |
| Improve performance        | Pick a better join strategy or partitioning |
| Debug slowness/skew        | Spot which stage is the problem             |


In [46]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("Query Plan").getOrCreate()
emtel_df = spark.read.csv('./data/testdata.csv', inferSchema=True, header=True)
print(emtel_df.rdd.getNumPartitions())  # Check the number of partitions
emtel_df.show(5)


8
+--------+--------------+-------+--------------------+--------------------+----------+-------------+--------+-------------+----------------+-------+---------------+--------------------+-------------------+-----------+--------+---------+--------+-----------------+------------------+-----------------+--------+--------+----------+----------+--------+--------+--------------+--------------+---------------+---------------+----------------+----------------+----------------+---------------+---------------+---------------+----------------------+----------+-------------+------------+--------------------+--------------------+-------------------+-------------------+----------------------------+---------------+---------------+--------------------+--------------------+-------------------+-------------------+------------------------+------------------------+--------+------------+----------------+----------------+----------------+-----------+-----------+-----------+------------+------------+--------

# Transformations: 
Business logic performed on dataframe.

**Types:**
- Wide Transformation: That requires shuffling of data.
- Narrow Transformation: Operations that do not involve shuffling of data.

## Narrow Transformation:
Operations that do not involve Shuffling.

E.g: Select, filter row where `NID_LEN` is greater than 5, adding new columns, selecting column, alter a existing colum, adding '234' to all values in `MSISDN` column.

In [82]:
cal_df = (
    emtel_df.select("MSISDN", "NID_LEN", "NID", "CUSTOMERIDNAME") # selecting specific columns
    .filter(F.col("NID_LEN") > 10)
    .withColumn("PHONE_NUMBER", F.concat(F.lit("234"), F.col("MSISDN"))) # adding a new number to all values in the MSISDN column
    .withColumn("MSISDN", F.length("MSISDN")) # altering an existing column
    .withColumn("FIRSTNAME", F.split("CUSTOMERIDNAME", " ").getItem(0)) # extracting the first name from CUSTOMERIDNAME
    .withColumn("LASTNAME", F.split("CUSTOMERIDNAME", " ").getItem(1)) # extracting the last name from CUSTOMERIDNAME
)
print(cal_df.rdd.getNumPartitions())

cal_df.show(5, False)
cal_df.explain(extended=True)  # Displaying the query plan with extended information

8
+------+-------+--------------+-----------------------------------+------------+---------+---------+
|MSISDN|NID_LEN|NID           |CUSTOMERIDNAME                     |PHONE_NUMBER|FIRSTNAME|LASTNAME |
+------+-------+--------------+-----------------------------------+------------+---------+---------+
|8     |14     |J0711874907942|WENSLEY JEAN                       |23454924133 |WENSLEY  |JEAN     |
|8     |14     |B160985220041C|MARIE ANGELO BOODHRAM              |23454846497 |MARIE    |ANGELO   |
|8     |14     |D090290300772B|J SEBASTIEN NICOLAS DURAU          |23457113437 |J        |SEBASTIEN|
|8     |14     |N110561330075F|PERINE FARIDA                      |23458468805 |PERINE   |FARIDA   |
|8     |14     |P0502893808515|MARY-JANE ISABELLE PATIENT-BABYLONE|23457119226 |MARY-JANE|ISABELLE |
+------+-------+--------------+-----------------------------------+------------+---------+---------+
only showing top 5 rows

== Parsed Logical Plan ==
'Project [MSISDN#9778, NID_LEN#9108, N

# Wide Transformation:

### Partitioning and Repartitioning Works in Spark

When you read a CSV file using:

```python
df = spark.read.csv("path/to/file.csv")
```

Spark will **initially partition the data** based on **file size and configuration settings**.

### What determines the number of partitions?

* **Number and size of input files** (each file block is typically one partition)
* **`spark.sql.files.maxPartitionBytes`** (default: 128 MB) → controls how large each partition can be.
* **`spark.sql.files.openCostInBytes`** (default: 4 MB) → cost Spark assumes for opening a file.
* These affect how Spark splits large files into partitions using a cost-based algorithm.

> 🔹 For example, a 1 GB file with 128 MB `spark.sql.files.maxPartitionBytes` → \~8 partitions

---

## 🔹 Repartitioning

You can explicitly change the number of partitions using:

```python
df.repartition(100)
```

## 🔹 When Should You Repartition?

* **Too few partitions?** → Not enough parallelism → poor CPU utilization.
* **Too many small partitions?** → Overhead from task scheduling and I/O.

Good rule of thumb: aim for **100–200 MB per partition** for optimal performance.

---

### 🔸 Example Scenario

```python
df = spark.read.csv("s3://bucket/large.csv")
println(df.rdd.getNumPartitions)  # See how many partitions Spark created

repartitioned = df.repartition(100)
print(repartitioned.rdd.getNumPartitions)  # Should print 100

repartitioned.write.parquet("output/")
```

In this example:

* `read.csv` determines initial partitions
* `repartition(100)` forces a shuffle to balance data across 100 partitions
* Then Spark will launch 100 tasks (1 per partition) in the next stage

**Note:** 
- repartitioning after an action operation, will cause another repartition.
- Repartition is a wide transformation that involves shuffling.


In [103]:
# Reparitioning: used to change the number of partitions in a DataFrame. Redistibute data across a different number of partitions.
# This can be useful for optimizing performance, especially before writing data to disk or performing operations that benefit from a specific partitioning scheme.

## get the number of partitions
print(cal_df.rdd.getNumPartitions())

repartitioned_df = cal_df.repartition(20)
print(repartitioned_df.rdd.getNumPartitions())

repartitioned_df.explain(extended=True)


8
20
== Parsed Logical Plan ==
Repartition 20, true
+- Project [MSISDN#9778, NID_LEN#9108, NID#9107, CUSTOMERIDNAME#9109, PHONE_NUMBER#9772, FIRSTNAME#9784, split(CUSTOMERIDNAME#9109,  , -1)[1] AS LASTNAME#9791]
   +- Project [MSISDN#9778, NID_LEN#9108, NID#9107, CUSTOMERIDNAME#9109, PHONE_NUMBER#9772, split(CUSTOMERIDNAME#9109,  , -1)[0] AS FIRSTNAME#9784]
      +- Project [length(cast(MSISDN#9106 as string)) AS MSISDN#9778, NID_LEN#9108, NID#9107, CUSTOMERIDNAME#9109, PHONE_NUMBER#9772]
         +- Project [MSISDN#9106, NID_LEN#9108, NID#9107, CUSTOMERIDNAME#9109, concat(234, cast(MSISDN#9106 as string)) AS PHONE_NUMBER#9772]
            +- Filter (NID_LEN#9108 > 10)
               +- Project [MSISDN#9106, NID_LEN#9108, NID#9107, CUSTOMERIDNAME#9109]
                  +- Relation [MSISDN#9106,NID#9107,NID_LEN#9108,CUSTOMERIDNAME#9109,EMAIL#9110,GROSS_DATE#9111,CUSTOMER_TYPE#9112,USERTYPE#9113,SUBSCRIBERCAT#9114,SUBSCRIBERSUBCAT#9115,SEGMENT#9116,POSTPAID_TARIFF#9117,POSTPAID_MAINPROD

## COALESCE
Coalesce is used to explicitly reduce the number of partitions.


### 🔹 Example: Coalescing and CPU Usage

#### Cluster Setup:

* 3 Executors
* 4 CPU cores per executor (total: 12 cores)
* 10 initial partitions → 10 tasks → up to 10 cores used

#### After `df.coalesce(3)`:

* Spark merges partitions into 3: `P1_new`, `P2_new`, `P3_new`
* 3 tasks created (1 per new partition)
* **Only 3 CPU cores used** (1 per task)
* **9 cores remain idle** during that stage

---

### ✅ Key Points:

* `coalesce(n)` reduces partitions **without full shuffle**
* Spark tries to merge partitions **locally per executor**
* Fewer tasks = **less parallelism**
* Good for **reducing small files**, but **can underutilize CPUs**
* When it requires shuffling is when partition needs to be move from another executor in other to be merged to form the total number of partitions `df.coalesce(2)` acrros cluster.
* When you do a `.repartition(n)` a partitioning scheme is involved e.g, `RoundRobinPartitioning`. But `.coalesce(n)` doesn't involve a Partitioning scheme, because there is likely no shuffle.



In [119]:
repartitioned_df.rdd.getNumPartitions()
repartitioned_df.coalesce(3).explain(extended=False) # Coalesce reduces the number of partitions without a shuffle
# d.explain(extended=True) 

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Coalesce 3
   +- Exchange RoundRobinPartitioning(20), REPARTITION_BY_NUM, [plan_id=2584]
      +- Project [length(cast(MSISDN#9106 as string)) AS MSISDN#9778, NID_LEN#9108, NID#9107, CUSTOMERIDNAME#9109, concat(234, cast(MSISDN#9106 as string)) AS PHONE_NUMBER#9772, split(CUSTOMERIDNAME#9109,  , -1)[0] AS FIRSTNAME#9784, split(CUSTOMERIDNAME#9109,  , -1)[1] AS LASTNAME#9791]
         +- Filter (isnotnull(NID_LEN#9108) AND (NID_LEN#9108 > 10))
            +- FileScan csv [MSISDN#9106,NID#9107,NID_LEN#9108,CUSTOMERIDNAME#9109] Batched: false, DataFilters: [isnotnull(NID_LEN#9108), (NID_LEN#9108 > 10)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/c:/Users/nwachwi/Documents/per/spark-tutorial/spark-with-jupyter..., PartitionFilters: [], PushedFilters: [IsNotNull(NID_LEN), GreaterThan(NID_LEN,10)], ReadSchema: struct<MSISDN:int,NID:string,NID_LEN:int,CUSTOMERIDNAME:string>




# Read `Join` Physical Plan

SortMergeJoin: useful for large datasets
BroadcastHashJoin: for small datasets

join is a wide transformation, which involves shuffling of datasets.


In [127]:
# change the method of join from broadcast to sort merge join
# This is useful when dealing with large datasets where broadcast joins may not be efficient.
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")  # Disable broadcast joins for this example

emtel_df1 = emtel_df
emtel_df2 = emtel_df

joined = emtel_df1.join(
    emtel_df2,
    how="inner",
    on=emtel_df1["MSISDN"] == emtel_df2["MSISDN"]
)

joined.explain(extended=False)  # Displaying the query plan for the join operation
joined.show(3)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [MSISDN#9106], [MSISDN#22334], Inner
   :- Sort [MSISDN#9106 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(MSISDN#9106, 200), ENSURE_REQUIREMENTS, [plan_id=3204]
   :     +- Filter isnotnull(MSISDN#9106)
   :        +- FileScan csv [MSISDN#9106,NID#9107,NID_LEN#9108,CUSTOMERIDNAME#9109,EMAIL#9110,GROSS_DATE#9111,CUSTOMER_TYPE#9112,USERTYPE#9113,SUBSCRIBERCAT#9114,SUBSCRIBERSUBCAT#9115,SEGMENT#9116,POSTPAID_TARIFF#9117,POSTPAID_MAINPRODUCT#9118,POSTPAID_SUBPRODUCT#9119,NATIONALITY#9120,AON_DAYS#9121,AON_MONTH#9122,AON_YEAR#9123,ALTERNATE_MOB_NUM#9124,FAVOURITE_LOCATION#9125,DAYS_INACT  BAL_1#9126,BAL_2#9127,BAL_3#9128,DATA_USERS#9129,... 50 more fields] Batched: false, DataFilters: [isnotnull(MSISDN#9106)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/c:/Users/nwachwi/Documents/per/spark-tutorial/spark-with-jupyter..., PartitionFilters: [], PushedFilters: [IsNotNull(MSISDN)], ReadSchema: 

✅ What is SortMergeJoin in Spark?

**SortMergeJoin** is a join strategy Spark uses when:
- Both datasets are large
- Join keys are not suitable for broadcasting
- The data needs to be shuffled and sorted by the join keys

🔹 How it works:
- Shuffle both DataFrames based on the join key (hashPartitioning)
- Sort each partition by the join key
- Merge the sorted partitions together (like the merge step in merge sort) to find matching rows.




**hashPartitioning** is a shuffling strategy used during operations like joins, groupBy, and repartitioning.

**Here's how it works:**
Spark computes a hash for the partition key (e.g. MSISDN), then applies a modulo operation (%) using the number of shuffle partitions (default = 200).

This determines which partition (out of N) each row should go to.

# GroupBy

In [154]:
# joined.printSchema()

joined_df = (
    joined
    .groupBy("USERTYPE")
)